# Migrate SQL Server Database to Azure SQL VM

## Source SQL Instance
The following code is used to specify the source SQL Server instance.  Data and Server objects will be copied from this server to the target SQL Server instance.  In the following code cell set the following parameters:

*Note: the notebook currently is setup for SQL Authentication.  Future updates will add support for multiple authentication types.*

|Parameter|Description|
|---|---|
|sourceServerName| The name or IP address of the source instance|
|sourceLogin| sql login to connect to source instance with |

*Note: source password should be set in the environment variable SQLMIG_SourcePassword.  This is to avoid persisting the environment variable in the notebook file.*

Edit the code below to specify the above parameters to test connectivity to the source instance.




In [ ]:
$sourceServerName = '<server_name>'
$sourceLogin = '<user_name>'

$env:SQLMIG_SourcePassword = '<user_pass>'

## PowerShell Environment 
$sourceLoginPassword = ConvertTo-SecureString $env:SQLMIG_SourcePassword -AsPlaintext -Force
$sourceCredential = New-Object System.Management.Automation.PSCredential ('<user_name>', $sourceLoginPassword)
$sourceTest = Test-DbaConnection -SqlInstance $sourceServerName -SqlCredential $sourceCredential
$sourceTest
$sourceConnection = Connect-DbaInstance -SqlInstance $sourceServerName -SqlCredential $sourceCredential

## Target SQL Instance
The following code is used to specify the target SQL Server instance.  This is the SQL Server instance that Data and Server objects will be copied to.  In the following code cell set the following parameters:

*Note: the notebook currently is setup for SQL Authentication.  Future updates will add support for multiple authentication types.*

|Parameter|Description|
|---|---|
|targetServerName| The name or IP address of the target instance|
|targetLogin| sql login to connect to target instance with |

*Note: the target login password should be set in the environment variable SQLMIG_TargetPassword.  Thisis to avoid persisting the environment variable in the notebook file.*

Edit the code below to specify the above parameters to test connectivity to the target instance.

In [ ]:
$targetServerName = 'sqlmig.westus2.cloudapp.azure.com'
$targetLogin = 'cloudsa'

## TEMP - REMOVE BEFORE PUSHING CHANGES
$env:SQLMIG_TargetPassword = '<user_pass>'

## PowerShell Environment 
$targetLoginPassword = ConvertTo-SecureString $env:SQLMIG_TargetPassword -AsPlaintext -Force
$targetCredential = New-Object System.Management.Automation.PSCredential ('<user_name>', $targetLoginPassword)
$targetTest = Test-DbaConnection -SqlInstance $targetServerName -SqlCredential $targetCredential
$targetTest
$targetConnection = Connect-DbaInstance -SqlInstance $targetServerName -SqlCredential $targetCredential

## Login to Microsoft Azure
To configure and provision resources you must log into your Azure account and set the current subscription that is being used for the target SQL Server instance.  The following code will help you connect your account and choose the correct subscription.  When presented with the list of subscriptions, click on the desired subscription and press OK.

In [ ]:
#Connect-AzAccount
$migrationSubscription = Get-AzSubscription | Select-Object -Property Name, Id | Out-GridView -PassThru
Set-AzContext -SubscriptionId $migrationSubscription.Id

## Verify No Active Connections

In [ ]:
#TODO - filter connected proceesses for user connections


Get-DbaProcess -SqlInstance $SourceServerName -SqlCredential $sourceLogin | 
Select Host, login, Program

## Temporary Storage for Data Movement

Offline data migration attempts to use backup to URL and restore from URL as the mechanism for moving data from the source instance to the target instance.  This code will check existance of the specified storage account and container to use for data migration. If the resources do not exist they will be created.

In [ ]:
$resourceGroup = "sqlmig"
$blobStorageAccount = "tempsqlmigstorage"
$containerName = "backups"
$location = "West US 2"

# Storage Account
$storageAccount = Get-AzStorageAccount -ResourceGroupName $resourceGroup -Name $blobStorageAccount
if ($storageAccount -eq $null)
{
    # specified storage account does not yet exist, attempt to create it
    $storageAccount = New-AzStorageAccount -ResourceGroupName $resourceGroup -Name $blobStorageAccount -Location $location -SkuName Standard_LRS -Kind StorageV2
}
$storageAccount

# Container
$storageContext = $storageAccount.Context
$storageContainer = Get-AzStorageContainer -Name $containerName -Context $storageContext
if ($storageContainer -eq $null)
{
    #specified storage container does not yet exist, attempt to create it
    $storageContainer = New-AzStorageContainer -Name $containerName -Context $storageContext -Permission Container
}
$storageContainer

# Provide source instance with SAS token for blob access
$sourceSAS = (New-AzStorageAccountSASToken -Service Blob -ResourceType Object -Permission "rw" -Context $storageContext).TrimStart('?')
$sourceCred = New-DbaCredential -SqlInstance $sourceConnection -Name "https://$blobStorageAccount.blob.core.windows.net/$containerName" -Identity "SHARED ACCESS SIGNATURE" -SecurePassword (ConvertTo-SecureString $sourceSAS -AsPlainText -Force) -Force
$sourceCred

$targetSAS = (New-AzStorageAccountSASToken -Service Blob -ResourceType Object -Permission "rw" -Context $storageContext).TrimStart('?') # -ResourceType Container,Object
$targetCred = New-DbaCredential -SqlInstance $targetConnection -Name "https://$blobStorageAccount.blob.core.windows.net/$containerName" -Identity "SHARED ACCESS SIGNATURE" -SecurePassword (ConvertTo-SecureString $targetSAS -AsPlainText -Force) -Force
$targetCred



## Copy Databases to Target Server

The following code will present a list of the databases from the source SQL Server instance.  Select the list of databases to copy and press OK.  The Copy-DbaDatabase CMDLET will take a backup of each database using the azure storage account information above.  Each database backup will then be restored from the blob storage account.  The database restore will use server defaults for database file location and structure.

If the database being restored already exists on the target instance, the *Force* parameter determines the behavior of the notebook:

|Force|Description|
|---|---|
|$true| Overwrite the existing database on the target instance|
|$false| Do not restore the database if it already exists on the target instance|

*Note: This can be a very long running process based on the size of the databases being copied.  The notebook should be allowed to run until the CMDLET completes.*



In [ ]:
$databasesToCopy = Get-DbaDatabase -SqlInstance $sourceConnection | Select-Object -Property Name | Out-GridView -PassThru
$databaseList = New-Object System.Collections.ArrayList
foreach ($db in $databasesToCopy)
{
    $databaseList.Add($db.Name)    
}

$copyDatabaseParams = @{
    Database = $databaseList
    Source = $sourceConnection
    Destination = $targetConnection
    BackupRestore = $true
    SharedPath =  "https://$blobStorageAccount.blob.core.windows.net/$containerName"
    Force = $true
    Verbose = $false     
}

Copy-DbaDatabase @copyDatabaseParams